In [1]:
import random
import numpy as np
import math
import pandas as pd

In [2]:
with open('rowZip.csv', 'r') as file:
    rows = file.readlines()
rows = [[int(st) for st in row.split(',')] for row in rows]
rows[0]

[12,
 17,
 40,
 42,
 69,
 176,
 216,
 320,
 358,
 439,
 445,
 493,
 518,
 540,
 547,
 571,
 580,
 590,
 596,
 641,
 694,
 706,
 755,
 794]

In [3]:
with open('colZip.csv', 'r') as file:
    cols = file.readlines()
cols = [[int(st) for st in col.split(',')] for col in cols]
cols[0]

[4, 5, 6, 8, 9, 11, 14, 18, 39, 51, 58, 70, 85, 136]

def user_ranstr(num):
    userID = ''
    H = 'AB'
    userID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        userID += random.choice(H)
    return userID


def item_ranstr(num):
    itemID = ''
    H = 'def'
    itemID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        itemID += random.choice(H)
    return itemID

In [4]:
def initial_dataset(n,num,num2):
    #n: grid width, num: length of user/item ID , num2: max_bicluster_width
    bicluster = []
    for i in range(n*n):
        '''num_user = random.randrange(num2)
        num_item = random.randrange(num2)
        userID = []
        itemID = []
        j = 0
        k = 0
        while j < num_user:
            try:
                user = user_ranstr(num)
                userID.index(user)
            except:
                userID.append(user)
                j= j + 1 
        while k < num_item:
            try:
                item = item_ranstr(num)
                itemID.index(item)
            except:
                itemID.append(item)
                k = k + 1'''
        userID = rows[i]
        itemID = cols[i]
        single_set = [userID,itemID]
        bicluster.append(single_set)
    return bicluster

In [5]:
def reward_func(set1,set2):   
    intersection = 0
    for i in range(len(set2)):
        try:
            set1.index(set2[i])
            intersection = intersection + 1
     #       print(set2[i])
        except:
            pass 
   ## print("交:",intersection)
   # print("並:",(len(set1)+len(set2)-intersection))
    try:
        reward = intersection/(len(set1)+len(set2)-intersection)
    except:
        reward = 1
    return round(reward,3)

In [6]:
def gen_gridworld(width, data):
    left_num = width*width
    gridworld = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            num = random.randrange(left_num)
            gridworld[i][j] = data[num]
            data.remove( data[num])
            left_num = left_num - 1
    return gridworld

In [7]:
def gen_reward_grid(gridworld,width):
    grid_reward = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            reward=[]
            #print("i=",i,"j=",j)
            if j%width == width-1:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)       
            elif j%width == 0:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
            else: 
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
            grid_reward[i][j] = reward
    return grid_reward

In [8]:
def total_reward(grid_reward):
    return np.sum(grid_reward)

In [9]:
def hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta,iteration):
    i = random.randrange(width)
    j = random.randrange(width)
    i2 = random.randrange(width)
    j2 = random.randrange(width)
    data_grid[i][j], data_grid[i2][j2] =  data_grid[i2][j2], data_grid[i][j]
    reward_update(reward_grid,data_grid,i,j, width)
    reward_update(reward_grid,data_grid,i2,j2, width)
    new_tot_reward = total_reward(reward_grid)
    P_k = 1.0-beta*alpha**iteration
    #print("P_k:",P_k)
    backfire = random.random()
   # print("backfire:",backfire)
    T = 10000000
    flag = (iteration % T == 0)
    if P_k < backfire:
        if flag:
            print("backfire,old:",tot_reward," new:",new_tot_reward)
        return new_tot_reward, 1
    else: 
        if new_tot_reward > tot_reward:
            if flag:
                print("update:",new_tot_reward)
            return new_tot_reward, 1
        else:
            if flag:
                print("non_update:",tot_reward," new:F",new_tot_reward)
            data_grid[i][j],  data_grid[i2][j2] =   data_grid[i2][j2],  data_grid[i][j]
            reward_update(reward_grid,data_grid,i,j, width)
            reward_update(reward_grid,data_grid,i2,j2, width)
            return tot_reward, 0

In [10]:
def reward_update(grid_reward,gridworld,x,y, width):
    x_update = [x-1,x,x+1,x,x]
    y_update = [y,y,y,y-1,y+1]
    for i, j in zip(x_update, y_update):
        reward=[]
        #print("i=",i,"j=",j)
        if i < 0 or i > width-1 or j < 0 or j> width-1:
           # print("i=",i,"j=",j)
            continue
        if j%width == width-1:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)       
        elif j%width == 0:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
        else: 
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
        grid_reward[i][j] = reward

In [11]:
width = 20
id_length = 2
max_bicluster_width = 20
beta = 0.4
stop_value = 10000

data = initial_dataset(width,id_length,max_bicluster_width)
data_grid = gen_gridworld(width, data)
reward_grid = gen_reward_grid(data_grid,width)
tot_reward = total_reward(reward_grid)
print(tot_reward)
stop = 0
alpha = 0.99999
iteration = 0

742.904


In [12]:
stop_value

10000

In [ ]:
while True:
    iteration = iteration + 1
    tot_reward, update_bool = hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta, iteration)
    P_k = 1.0-beta*alpha**iteration
    if update_bool == 1:
        stop = 0
    else:
        stop = stop+1
    if iteration % 10000 == 0:
        print(P_k, tot_reward)
    if 1 - P_k < 1e-7:
        break
    #print("real: ", total_reward(gen_reward_grid(data_grid,width)))
print(tot_reward)

0.6380652137540963 804.0219999999999
0.67250802626283 794.968
0.7036731562204706 787.8140000000001
0.7318725178443317 795.678
0.7573883426483397 807.816
0.780476004139156 817.244
0.8013665737056469 805.316
0.8202691332821546 816.662
0.8373728679317141 840.034
0.8528489592927013 831.412
0.866852298839354 823.47
0.8795230381032203 842.786
0.8909879913708328 853.0540000000001
0.9013619048964144 864.2919999999999
0.9107486053324516 854.8399999999999
0.9192420388721302 874.402
0.9269272115038236 872.578
0.9338810397881107 877.9939999999999
0.9401731206722711 880.452
0.9458664280468974 860.374
0.9510179430160663 895.4259999999999
0.9556792241890882 908.13
0.9598969237015627 916.114
0.9637132541303047 919.562
0.967166410975231 923.174
0.970290954936586 925.164
0.9731181578135083 931.144
0.97567631548584 953.7840000000001
0.9779910311116368 953.488
0.9800854713747438 966.2
0.981980598347075 956.6020000000001
0.9836953792861737 972.7440000000001
0.9852469764678018 992.642
0.9866509189534827 986

In [ ]:
total_reward(gen_reward_grid(data_grid,width))

In [ ]:
data_grid[0][1]

In [ ]:
import pickle

f1 = open("dic", "rb")
f2 = open("dic_inv", "rb")
dic = pickle.load(f1)
dic_inv = pickle.load(f2)
f1.close()
f2.close()

In [ ]:
real_grid = [[[data_grid[k][j][0], [dic_inv[i] for i in data_grid[j][k][1]]] for j in range(width)] for k in range(width)]

In [ ]:
f = open("grid", "wb")
pickle.dump(real_grid, f)
f.close()

In [ ]:
f = open("grid", "rb")
pickle.load(f) == real_grid
f.close()